Coded by Lujia Zhong @lujiazho<br>
Reference: https://github.com/tornadomeet/ResNet, https://github.com/weiaicunzai/pytorch-cifar100

In [1]:
import time
import torch
import torch.nn as nn

# resnet 18 and resnet 34
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut = nn.Sequential()

        if stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        return nn.ReLU()(self.residual_function(x) + self.shortcut(x))


# resnet over 50
class BottleNeck(nn.Module):
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * 4),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * 4:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * 4, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * 4)
            )

    def forward(self, x):
        return nn.ReLU()(self.residual_function(x) + self.shortcut(x))


class ResNet(nn.Module):

    def __init__(self, config):
        super().__init__()
        
        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, stride=2, kernel_size=7, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.conv2_x = self.conv_layer(config.block, 64, config.num_block[0], ini_stride=1)
        self.conv3_x = self.conv_layer(config.block, 128, config.num_block[1], ini_stride=2)
        self.conv4_x = self.conv_layer(config.block, 256, config.num_block[2], ini_stride=2)
        self.conv5_x = self.conv_layer(config.block, 512, config.num_block[3], ini_stride=2)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * config.block.expansion, config.num_classes)

    def conv_layer(self, block, out_channels, num_blocks, ini_stride):
        layers = [block(self.in_channels, out_channels, ini_stride)]
        self.in_channels = out_channels * block.expansion
        
        for _ in range(num_blocks-1):
            layers.append(block(self.in_channels, out_channels, 1))
            
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.max_pool(output)
        
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        
        output = self.avg_pool(output).view(output.shape[0], -1)
        output = self.fc(output)
        output = nn.functional.softmax(output, dim=-1)

        return output

class ResnetConfig:
    def __init__(self, ResnetType='18'):
        self.block = None
        self.num_classes = 100
        
        if ResnetType == '18' or ResnetType == '34':
            self.block = BasicBlock
        else:
            self.block = BottleNeck
        
        if ResnetType == '18':
            self.num_block = [2, 2, 2, 2]
        elif ResnetType == '34':
            self.num_block = [3, 4, 6, 3]
        elif ResnetType == '50':
            self.num_block = [3, 4, 6, 3]
        elif ResnetType == '101':
            self.num_block = [3, 4, 23, 3]
        else:
            self.num_block = [3, 8, 36, 3]


img_size = torch.randint(100, 400, (1,)).item()
model = ResNet(ResnetConfig(ResnetType='50'))

In [2]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

iterarions = 2
begin = time.time()

# Training
for iterarion in range(iterarions):
    x = torch.rand((4, 3, img_size, img_size))
    y = torch.randint(0, 100, (4,))
    
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)

    if iterarion % 1 == 0:
        print('Iterarion:', '%2d,' % (iterarion + 1), 'loss =', '{:.4f}'.format(loss))

    loss.backward()
    optimizer.step()

print(f"{(time.time() - begin)/iterarions:.4f}s / iterarion")

Iterarion:  1, loss = 4.6074
Iterarion:  2, loss = 4.6054
1.4143s / iterarion
